# Creación de librería para la selección de portafolios.

## Objetivos.

  ###### Objetivo General: 
 > Realizar una librería con varias funciones útiles para la selección de portafolios.

###### Objetivos Específicos:
> Investigar los comandos que contiene la librería CVXOPT.

> Proponer la nueva librería para la selección de portafolios.

> Resolver problemas de selección de portafolios una vez teniendo la librería.

> Comparar el trabajo realizado con las funciones minimize con CVXOPT.

## Descripción.

Primero, se tiene que investigar qué es la librería CVXOPT, como funciona y cuales son los componentes:

-  CVXOpt es un paquete gratuito que se utiliza para la optimización convexa. Su principal propósito, es conseguir el desarrollo de software para aplicaciones de optimización convexa mediante la construcción de una librería estándar extensa de Python y empleando las fortalezas de Python como lenguaje de programación de alto nivel.
- CVXOpt fue desarrollado inicialmente por Martin Andersen, Joachim Dahl y Lieven Vandenberghe para emplearlo en su propio trabajo (Bernarbé, 2013).

**¿Qué es la optimización convexa?  **

La optimización convexa trata el problema general de minimizar una función convexa, donde una función es convexa si su dominio $D$ ⊂ $R^n$ es convexo para todo $x, y$ ∈ $D$, para todo λ ∈ [0,1], se verifica:

$$f(λx + (1-λ)y) ≤ λf (x) + (1 − λ)f (y).$$

Donde por ejemplo: $ f(x) = x^2 $ es convexa. (Berrendero, s.f.)

Para instalar la librería, se utiliza el siguiente comando:

###### conda install -c conda-forge cvxopt

In [31]:
import cvxopt as opt
import cvxopt.solvers as optsolvers

In [14]:
help(opt)

Help on package cvxopt:

NAME
    cvxopt - Python package for convex optimization

DESCRIPTION
    CVXOPT is a free software package for convex optimization based on the 
    Python programming language. It can be used with the interactive Python 
    interpreter, on the command line by executing Python scripts, or 
    integrated in other software via Python extension modules. Its main 
    purpose is to make the development of software for convex optimization 
    applications straightforward by building on Python's extensive standard 
    library and on the strengths of Python as a high-level programming 
    language.

PACKAGE CONTENTS
    _version
    amd
    base
    blas
    cholmod
    coneprog
    cvxprog
    dsdp
    fftw
    glpk
    gsl
    info
    lapack
    misc
    misc_solvers
    modeling
    msk
    printing
    solvers
    umfpack

CLASSES
    builtins.object
        cvxopt.base.matrix
        cvxopt.base.spmatrix
    
    class matrix(builtins.object)
     |  Metho

In [48]:
help(optsolvers)

Help on module cvxopt.solvers in cvxopt:

NAME
    cvxopt.solvers - Convex optimization solvers.

DESCRIPTION
    conelp:   solves linear cone programs.
    coneqp:   solves quadratic cone programs.
    cp:       solves nonlinear convex problem.
    cpl:      solves nonlinear convex problems with linear objectives.
    gp:       solves geometric programs.
    lp:       solves linear programs.
    qp:       solves quadratic programs.
    sdp:      solves semidefinite programs.
    socp:     solves second-order cone programs.
    options:  dictionary with customizable algorithm parameters.

FUNCTIONS
    conelp(c, G, h, dims=None, A=None, b=None, primalstart=None, dualstart=None, kktsolver=None, xnewcopy=None, xdot=None, xaxpy=None, xscal=None, ynewcopy=None, ydot=None, yaxpy=None, yscal=None, **kwargs)
        Solves a pair of primal and dual cone programs
        
            minimize    c'*x
            subject to  G*x + s = h
                        A*x = b
                        s 

## Solución del problema: Creación de la librería.

In [24]:
def Er(w):
    er = Eind.dot(w)
    return er

In [117]:
def sp(w, Sigma):
    sp = w.dot(Sigma).dot(w)
    return sp

In [118]:
def sharpe(er,sp,rf):
    s = (er-rf)/sp
    return s

In [119]:
def max_sharpe(w,Sigma,rf,Eind):
    Erp = Eind.dot(w)
    varp = w.dot(Sigma).dot(w)
    return -(Erp-rf)/np.sqrt(varp)

In [120]:
def min_var(cov, allow_short=False):
    n = len(cov)
    cov_val = opt.matrix(cov.values)
    ceros = opt.matrix(0.0, (n, 1))

    if not allow_short:
        # x >= 0
        mat_idt = opt.matrix(-np.identity(n)) #matriz de identidad 
        ceros_mat_idt = opt.matrix(0.0, (n, 1))
    else:
        G = None
        h = None

    a = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    optsolvers.options['show_progress'] = False
    # qp(P, q, G=None, h=None, A=None, b=None, solver=None, kktsolver=None, initvals=None, **kwargs)
    # Se resuelve una función cuadrática
    sol = optsolvers.qp(cov_val, ceros, mat_idt, ceros_mat_idt, a, b)

    if sol['status'] != 'optimal':
        warnings.warn("Convergence problem")

    w = pd.Series(sol['x'], index=cov.index)
    return w

In [29]:
def optimal_portfolio(daily_ret, n_opt, risk_free):
    # Frontier points
    #Packages
    import pandas as pd
    import sklearn.covariance as skcov
    import numpy as np
    import cvxopt as opt
    from cvxopt import blas, solvers
    num_stocks = daily_ret.columns.size   
    #cvxopt matrices
    robust_cov_matrix = skcov.ShrunkCovariance().fit(daily_ret).covariance_
    S = opt.matrix(robust_cov_matrix)
    daily_ret_mean = daily_ret.mean().values
    mus = np.linspace(daily_ret_mean.min(), daily_ret_mean.max(), n_opt)
    # Constraint matrices
    G = -opt.matrix(np.concatenate((np.array([daily_ret_mean]),np.eye(num_stocks)),axis=0))
    p = opt.matrix(np.zeros((num_stocks, 1)))
    A = opt.matrix(np.ones((1,num_stocks)))
    b = opt.matrix(np.array([1.0]))    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = np.zeros((n_opt, num_stocks))
    for k in range(n_opt):
        h = -opt.matrix(np.concatenate((np.array([[mus[k]]]),np.zeros((num_stocks,1))), axis=0))
        portfolios[k,:] = np.asarray(solvers.qp(S, p, G, h, A, b)['x']).T[0]
    # Risk and returns
    returns = 252*portfolios.dot(daily_ret_mean)
    risks = np.zeros(n_opt)
    for i in range(n_opt):
        risks[i] = np.sqrt(252*portfolios[i,:].dot(robust_cov_matrix).dot(portfolios[i,:].T))
    sharpe = (returns-risk_free)/risks
    return  pd.DataFrame(data=np.column_stack((returns,risks,sharpe,portfolios)),
                         columns=(['Rendimiento','SD','Sharpe']+list(daily_ret.columns)))

In [99]:
annual_ret_summ = pd.DataFrame(columns=['Bonos', 'Acciones', 'Desarrollado', 'Emergente', 'Privados', 'Real'], index=['Media', 'Volatilidad'])
annual_ret_summ.loc['Media'] = np.array([0.0400, 0.1060, 0.0830, 0.1190, 0.1280, 0.0620])
annual_ret_summ.loc['Volatilidad'] = np.array([0.0680, 0.2240, 0.2210, 0.3000, 0.2310, 0.0680])

annual_ret_summ.round(4)

,Bonos,Acciones,Desarrollado,Emergente,Privados,Real
Media,0.04,0.106,0.083,0.119,0.128,0.062
Volatilidad,0.068,0.224,0.221,0.3,0.231,0.068


In [18]:
optimal_portfolio(annual_ret_summ,6,0.05)

     pcost       dcost       gap    pres   dres
 0:  1.0121e-03 -1.0961e+00  1e+00  1e-16  3e+00
 1:  1.0090e-03 -1.1952e-02  1e-02  2e-16  4e-02
 2:  8.0393e-04 -8.4958e-04  2e-03  5e-17  5e-03
 3:  2.4350e-04 -3.1637e-04  6e-04  2e-16  1e-18
 4:  1.5395e-04  5.8961e-05  9e-05  1e-16  4e-19
 5:  1.1646e-04  9.9288e-05  2e-05  1e-16  2e-19
 6:  1.0894e-04  1.0784e-04  1e-06  1e-16  3e-19
 7:  1.0818e-04  1.0817e-04  1e-08  1e-16  3e-19
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0248e-03 -1.0617e+00  1e+00  0e+00  3e+00
 1:  1.0217e-03 -1.1523e-02  1e-02  7e-17  4e-02
 2:  8.1897e-04 -7.7228e-04  2e-03  9e-17  5e-03
 3:  2.6563e-04 -2.5925e-04  5e-04  2e-16  1e-18
 4:  1.9544e-04  7.8599e-05  1e-04  2e-16  4e-19
 5:  1.7575e-04  1.5846e-04  2e-05  1e-16  3e-17
 6:  1.6624e-04  1.6564e-04  6e-07  2e-16  1e-17
 7:  1.6578e-04  1.6577e-04  6e-09  9e-17  6e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0376e-03 -1.0258e

,Rendimiento,SD,Sharpe,Bonos,Acciones,Desarrollado,Emergente,Privados,Real
0,15.305682,0.233506,65.333072,3.880180e-01,1.318637e-05,8.989931e-06,4.552759e-06,1.986008e-05,6.119354e-01
1,21.445227,0.289056,74.017688,1.318643e-01,3.138178e-05,3.703197e-06,3.332060e-06,1.881805e-01,6.799168e-01
2,29.282448,0.423324,69.054517,2.996200e-05,9.684597e-05,1.060440e-05,1.267620e-05,4.470575e-01,5.527924e-01
3,37.119603,0.611037,60.666670,7.459244e-07,2.324527e-06,4.023661e-07,3.218833e-07,7.187747e-01,2.812215e-01
4,44.956831,0.818645,54.855105,8.979440e-06,1.197918e-05,2.404294e-06,3.514165e-06,9.903782e-01,9.594900e-03
5,52.794000,1.392323,37.882002,-9.643971e-12,-1.652486e-11,1.178038e-11,1.000000e+00,2.172284e-11,2.100751e-12


## Resultados.

### Comparando la librería con la función *minimize*.

#### *Vamos a obtener los pesos del portafolio de mínima varianza de la siguiente matriz de correlación:*

In [121]:
import pandas as pd
import numpy as np
rf = 0.05
corr = pd.DataFrame(data= np.array([[1.0000, 0.5003, 0.4398, 0.3681, 0.2663],
                                    [0.5003, 1.0000, 0.5420, 0.4265, 0.3581],
                                    [0.4398, 0.5420, 1.0000, 0.6032, 0.3923],
                                    [0.3681, 0.4265, 0.6032, 1.0000, 0.3663],
                                    [0.2663, 0.3581, 0.3923, 0.3663, 1.0000]]))
data_num = len(corr.index)
corr.round(4)

,0,1,2,3,4
0,1.0000,0.5003,0.4398,0.3681,0.2663
1,0.5003,1.0000,0.5420,0.4265,0.3581
2,0.4398,0.5420,1.0000,0.6032,0.3923
3,0.3681,0.4265,0.6032,1.0000,0.3663
4,0.2663,0.3581,0.3923,0.3663,1.0000


In [122]:
w_MVP = min_var(corr)
w_MVP

0    0.254625
1    0.158473
2    0.091929
3    0.203185
4    0.291789
dtype: float64

In [123]:
Eind = np.array([0.1355, 0.1589, 0.1519, 0.1435, 0.1497])
Er_MVP = Er(Eind)
Er_MVP

0.10968541000000001

In [130]:
Vol = np.array([0.1535, 0.2430, 0.2324, 0.2038, 0.2298])
D = np.diag(Vol)
Sigma = D.dot(corr).dot(D)
sp_MVP = np.sqrt(sp(w_MVP,Sigma))
sp_MVP

0.15110391024466208

In [106]:
sharpe_MVP = sharpe(Er_MVP,sp_MVP,0.05)
sharpe_MVP

0.3949958005941707

*La librería minimize nos arroja los siguientes resultados: *

In [132]:
from scipy.optimize import minimize

w0 = np.ones(data_num)/data_num
bnds = ((0,1),)*data_num
cons = ({'type': 'eq', 'fun': lambda w: np.sum(w)-1},)

minvar = minimize(varianza,w0,args = (Sigma),bounds =bnds, constraints = cons)

w_minvar = minvar.x
Er_minvar = Eind.dot(w_minvar)
sp_minvar = np.sqrt(sp(w_minvar,Sigma))
sharpe_minvar = (Er_minvar-rf)/sp_minvar
w_minvar,Er_minvar,sp_minvar,sharpe_minvar


(array([0.61779704, 0.        , 0.        , 0.20939436, 0.1728086 ]),
 0.13962903703376398,
 0.13644692708365533,
 0.6568783845077918)

In [134]:
EMV = minimize(max_sharpe,w0,args=(Sigma,rf,Eind),bounds=bnds,constraints=cons)

w_EMV = EMV.x
Er_EMV = Eind.dot(w_EMV)
sp_EMV = np.sqrt(w_EMV.dot(Sigma).dot(w_EMV))
sharpe_EMV = (Er_EMV-rf)/sp_EMV
w_EMV,Er_EMV,sp_EMV,sharpe_EMV

(array([0.50714173, 0.07470887, 0.02471534, 0.18943972, 0.20399434]),
 0.14206575653037432,
 0.138561993746032,
 0.6644372965585362)

In [149]:
cov1 = w_MVP.dot(Sigma).dot(w_EMV)
cov2 = w_minvar.dot(Sigma).dot(w_EMV)
cov1,cov2

(0.02012289081103911, 0.018690275417792227)

In [150]:
corr1 = cov/(sp_MVP*sp_EMV)
corr2 = cov/(sp_minvar*sp_EMV)
corr1,corr2

(0.8926801430210148, 0.9885708904645416)

## Conclusiones.

## Referencias.

- Bernarbé, Jorge (2013). * Optimización en Python: * CVXOpt Recuperado de: https://www.pybonacci.org/2013/12/13/optimizacion-en-python-cvxopt/
- Berrendero, J. (s.f.) * Funciones convexas y optimización convexa * Recuperado de: http://verso.mat.uam.es/~joser.berrendero/cursos/Matematicas-IO/io-tema4-16.pdf

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Francisco Enriquez, Job Ramírez & Oscar Flores.
</footer>